In [13]:
import pandas as pd
import random 

In [24]:
def preprocessing():
    # read the csvs
    training_data = pd.read_csv(filepath_or_buffer="TREC_dataset/train.csv", sep=",")
    test_data = pd.read_csv(filepath_or_buffer="TREC_dataset/test.csv", sep=",") 
    
    # get all coarse labels, they are integers from 0 to 5
    labels_coarse = list(set([x for x in training_data["label-coarse"]]))
    
    # choosing two classes to merge
    choices = random.sample(labels_coarse, k=2)
    
    # define the new class mappings old coarse labels (key): new coarse labels (key)
    # new labels are 0, 1, 2, 3, 4, and 4 is OTHERS
    new_labels = {}
    
    curr = 0
    for i in labels_coarse:
        if i not in new_labels:
            if i not in choices:
                new_labels[i] = curr
                curr += 1
            else:
                new_labels[i] = 4
                
    # replace all the labels in the training and test dataset
    for i, row in training_data.iterrows():
        training_data.at[i,'label-coarse'] = new_labels[training_data.at[i,'label-coarse']]
        
    for i, row in test_data.iterrows():
        test_data.at[i,'label-coarse'] = new_labels[training_data.at[i,'label-coarse']]
        
    # save the files
    training_data.to_csv(path_or_buf="TREC_dataset/modified_training_data.csv", index=False, sep=',')
    test_data.to_csv(path_or_buf="TREC_dataset/modified_test_data.csv", index=False, sep=',')

In [25]:
preprocessing()